In [1]:
import pandas as pd
import plotly.express as px
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [108]:
def date_str_validate(date):
    """
    Validate date format from user's input
    """
    while True:
        #date = input("Enter the date (YYYY-MM-DD) or 'today' for today's date: ")
        if date == 'today':
            date = dt.date.today()
            print(date)
            return date
        try: 
            date = dt.date.fromisoformat(date)
            print(date)
            return date
        except:
            print("Incorrect data format, should be YYYY-MM-DD")
            continue

In [110]:
def cumulative_expenses():

    """
    THIS FUNCTION ASK USER FOR DATE, DESCRIPTION ON WHAT THEY'VE SPENT, AND AMOUNT OF EXPENSE.
    EACH ITEM WILL BE SORTED INTO CATEGORY AND STORED IN A DATAFRAME.
    """
    
    try:
        name = input("Your name: ")
        df = pd.read_csv(f"{name}.csv")  
    except:
        df = pd.DataFrame(columns=['date', 'description', 'category', 'amount'])
            
    while True:
        date = input("Enter the date (YYYY-MM-DD) or 'today': ")
        date_str_validate(date)

        desc = input("what have you spent on? ")
        if desc in ['cloth', 'watch', 'shoes', 'shirt', 'pants','skirt', 'dress', 'hat']:
            cat = 'Shopping'
        elif desc in ['lidl', 'kaufland', 'food', 'veggies', 'meat', 'eggs', 'milk']:
            cat = 'Food & Grocery'
        elif desc in ['meals', 'drink','coffee','bakery', 'cake']:
            cat = 'Bar & Restaurants'
        elif desc in ['bus', 'flight','train','taxi']:
            cat = 'Transport & Car'
        elif desc in ['rent', 'apartment']:
            cat = 'Rent'
        elif desc in ['stock', 'fund', 'bond', 'investment', 'invest', 'emergency fund','saving','Scalable Capital', 'Trade Republic']:
            cat = 'Savings & Investments'
        elif desc in ['withdraw', 'cash']:
            cat = 'Cash'
        elif desc in ['ATM']:
            cat = 'ATM'
        elif desc in ['accommodation','room','hostel','hotel', 'airbnb']:
            cat = 'Travel'
        elif desc in ['internet']:
            cat = 'Household & Utilities'
        elif desc in ['rossmann', 'dm']:
            cat = 'Healthcare & Drug'     
        elif desc in ['donation']:
            cat = 'Giving'       
        else:
            cat = 'Other'
            
        category = cat
        amount = float(input("How much did you spend? "))
        data = [[date, desc, category, amount]]
        new_df = pd.DataFrame(data, columns=['date','description','category','amount'])
    
        df = df.append(new_df, ignore_index = True)
        done = input('Type "done" when finish or "else" to add more items >> ')   
        if done == 'done':
            break   
        else:
            continue
            
    df.to_csv(f"{name}.csv", index=False)
    
    print("LAST FIVE EXPENSES: \n",df.tail(5),"\nGood day! "+name+"\nNEWLY ADDED ITEM(S): \n",new_df)

    return df, new_df, name


In [111]:
def analyze_expense(df, name):
    
    """
    THIS FUNCTION TAKE DATAFRAME AND SUMMARIZE THE CUSTUMER'S EXPENSES MONTHLY/YEARLY
    """
    
    df['date'] = pd.to_datetime(df['date']) #change data into the datetime type
    df['year_month'] = df['date'].dt.to_period('M').astype(str) #extract YYYY-MM 
    df['year'] = df['date'].dt.strftime('%Y') #extract year
    grouped_m = round(df.groupby(['year_month','category']).mean(),2).sort_values('amount',ascending=False) #calculate average amount each category
    grouped_y = round(df.groupby(['year','category']).mean(),2).sort_values(['year','amount'],ascending=True).reset_index() #calculate average amount/year
    grouped_y_sum = round(df.groupby(['year','category']).sum(),2).sort_values(['year','amount'],ascending=True).reset_index()
    cat_names = grouped_m.reset_index() #after calculate, we'll list the category names ranked by the amount of expenses
    cat_names = cat_names['category'].values.tolist() #get the category names
    
    # monthly expenses
    fig = px.pie(grouped_m, values='amount', 
                 names = cat_names, color=cat_names,
                 title="Average Monthly Expenses")
    fig.show()
    
    # total expenses (so far)
    fig = px.bar(df, x='year_month', y='amount', color='category',
                 labels={'amount':'Total Amount',
                        'year_month':'Date'},
                 title='Montly Total Expenses',)
    fig.show()

    # yearly expenses (monthly average)
    #fig = px.bar(grouped_y, x='year', y='amount', color='category',
    #            labels={'amount':'Amount', 'year':'Year'},
    #            title="Average Yearly Expenses by Category")
    #fig.show()
    
    
    # yearly expenses
    fig = px.bar(grouped_y_sum, x='year', y='amount', color='category',
                labels={'amount':'Amount', 'year':'Year'},
                title="Yearly Total Expenses by Category")
    fig.show()
    

In [114]:
def main():
    
    action = input("""
    Enter '1' to ADD expense \n
    Enter '2' to VIEW your expense:
    
    """)
    if action == '1':
        df, new_df, name = cumulative_expenses()
        analyze_expense(df, name)
    else:
        name = input("Your name: ")
        df = pd.read_csv(f'{name}.csv')
        analyze_expense(df, name)

In [115]:
main()


    Enter '1' to ADD expense 

    Enter '2' to VIEW your expense:
    
    1
Your name: test
Enter the date (YYYY-MM-DD) or 'today': 2024-02-13
2024-02-13
what have you spent on? dm
How much did you spend? 1
Type "done" when finish or "else" to add more items >> done
LAST FIVE EXPENSES: 
           date description           category  amount
48  2024-02-13    notebook              Other    4.00
49  2024-02-13       water              Other    0.45
50  2024-02-13        book              Other    8.99
51  2024-02-13      coffee  Bar & Restaurants    1.00
52  2024-02-13          dm  Healthcare & Drug    1.00 
Good day! test
NEWLY ADDED ITEM(S): 
          date description           category  amount
0  2024-02-13          dm  Healthcare & Drug     1.0
